In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
os.chdir("/home/yw699/codes/LLM-Hallu")
sys.path.append(os.path.abspath("src"))
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"

In [ ]:
import yaml
import math
from dataset import Dataset
from prompt_engineer import PromptGenerator
from models import HuggingfaceModel
from samplers import Sampler
from utils import *
import logging

In [ ]:
setup_logger()

In [ ]:
with open("configs/experiment_config1.yaml", "r") as file:
    config = yaml.safe_load(file)
wandb_config = config["wandb"]


In [ ]:
import wandb
user = os.environ['USER']
slurm_jobid = os.getenv('SLURM_JOB_ID', None)
scratch_dir = os.getenv('SCRATCH_DIR', '.')
entity = os.getenv('WANDB_SEM_UNC_ENTITY', None)

dir = f"{scratch_dir}/{user}/{entity}"
if not os.path.exists(dir):
        os.makedirs(dir)
project = config["wandb"]["project"]

if config["wandb"]["debug"]:
    project = f"{project}_debug"

experiment_lot = config["wandb"]['experiment_lot']
notes=f'slurm_id: {slurm_jobid}, experiment_lot: {experiment_lot}'

wandb.init(
    entity=entity,
    project= project,
    dir=dir,
    config=config,
    notes=notes,
)

logging.info('Finished wandb init.')

In [ ]:
dataset_loader = Dataset(config)
train_dataset, validation_dataset = dataset_loader.load_data()


if not isinstance(train_dataset, list):
        logging.info('Train dataset: %s', train_dataset)

answerable_indices, unanswerable_indices = split_dataset(train_dataset)


if config["dataset"]['answerable_only']:
        unanswerable_indices = []
        val_answerable, val_unanswerable = split_dataset(validation_dataset)
        del val_unanswerable
        validation_dataset = [validation_dataset[i] for i in val_answerable]



In [ ]:
# prompt_indices = random.sample(answerable_indices, args.num_few_shot)
# experiment_details['prompt_indices'] = prompt_indices
# remaining_answerable = list(set(answerable_indices) - set(prompt_indices))
# # Create Few-Shot prompt.
# make_prompt = utils.get_make_prompt(args)
# BRIEF = utils.BRIEF_PROMPTS[args.brief_prompt]



# prompt = utils.construct_fewshot_prompt_from_indices(
# train_dataset, prompt_indices, BRIEF, arg, make_prompt)
# experiment_details['prompt'] = prompt
# experiment_details['BRIEF'] = BRIEF
# logging.info('Prompt is: %s', prompt)

In [ ]:
# promptgenerator = PromptGenerator(config,validation_dataset)

# prompt = promptgenerator.generate_prompt_by_id(1)
# len(promptgenerator.get_unused_indices())
# print(prompt)

# prompts =promptgenerator.generate_prompts_by_count(5)
# len(promptgenerator.get_unused_indices())